### Read in the CSV files

In [1]:
import pandas as pd
from IPython.core.display import display

batting = pd.read_csv('Batting.csv')
players = pd.read_csv('People.csv')

display(batting.head())
display(players.head())

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0


,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


### Merge the People and Batting DF

In [2]:
PB_DF = pd.merge(batting, players, on='playerID')
PB_DF.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,Abercrombie,Francis Patterson,NaN,NaN,NaN,NaN,1871-10-21,1871-10-21,aberd101,abercda01
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,Addy,Robert Edward,160.0,68.0,L,L,1871-05-06,1877-10-06,addyb101,addybo01
2,addybo01,1873,1,PH2,NaN,10,51,12,16,1,...,Addy,Robert Edward,160.0,68.0,L,L,1871-05-06,1877-10-06,addyb101,addybo01
3,addybo01,1873,2,BS1,NaN,31,152,37,54,6,...,Addy,Robert Edward,160.0,68.0,L,L,1871-05-06,1877-10-06,addyb101,addybo01
4,addybo01,1874,1,HR1,NaN,50,213,25,51,9,...,Addy,Robert Edward,160.0,68.0,L,L,1871-05-06,1877-10-06,addyb101,addybo01


### Add calculated columns

In [3]:
# Using the f" to format
year = PB_DF['birthYear'].apply(lambda x: f'{x:.0f}')
month = PB_DF['birthMonth'].apply(lambda x: f'{x:.0f}')
day = PB_DF['birthDay'].apply(lambda x: f'{x:.0f}')

# Concate birth year, month, day
dob =  year + "-" + month + "-" + day
# Add column with birth info
PB_DF['DOB'] = dob

PB_DF['fullName'] = PB_DF['nameFirst'].astype(str) + " " + PB_DF['nameLast'].astype(str)

PB_DF.drop(['birthYear', 'birthMonth', 'birthDay', 'nameFirst', 'nameLast', 'retroID', 'bbrefID'], axis=1, inplace=True)
# Drop rows that have all 'na'
# PB_DF = PB_DF.dropna(how='all')
PB_DF.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,deathCity,nameGiven,weight,height,bats,throws,debut,finalGame,DOB,fullName
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,Philadelphia,Francis Patterson,NaN,NaN,NaN,NaN,1871-10-21,1871-10-21,1850-1-2,Frank Abercrombie
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,Pocatello,Robert Edward,160.0,68.0,L,L,1871-05-06,1877-10-06,1842-2-nan,Bob Addy
2,addybo01,1873,1,PH2,NaN,10,51,12,16,1,...,Pocatello,Robert Edward,160.0,68.0,L,L,1871-05-06,1877-10-06,1842-2-nan,Bob Addy
3,addybo01,1873,2,BS1,NaN,31,152,37,54,6,...,Pocatello,Robert Edward,160.0,68.0,L,L,1871-05-06,1877-10-06,1842-2-nan,Bob Addy
4,addybo01,1874,1,HR1,NaN,50,213,25,51,9,...,Pocatello,Robert Edward,160.0,68.0,L,L,1871-05-06,1877-10-06,1842-2-nan,Bob Addy


### Show all the players that have played 50+ games and are still active

In [4]:
# Convert the dtype of the columns
PB_DF['finalGame'] = pd.to_datetime(PB_DF['finalGame'])
PB_DF['G'] = pd.to_numeric(PB_DF['G'])
# Set the variable for the boolean mask
start_date = '2020-01-01'
end_date = '2020-12-31'
min_games = 50
# Set the condition for the Dataframe and assign it to mask
mask = (PB_DF['finalGame'] > start_date) & (PB_DF['finalGame'] < end_date) & (PB_DF['G'] >= min_games)
# Create new Dataframe for current active players with > 50 career games
active_players = PB_DF.loc[mask]

display(active_players.head())
display(active_players.shape)
display(active_players['playerID'].nunique())

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,deathCity,nameGiven,weight,height,bats,throws,debut,finalGame,DOB,fullName
88118,pujolal01,2001,1,SLN,NL,161,590,112,194,47,...,NaN,Jose Alberto,235.0,75.0,R,R,2001-04-02,2020-08-08,1980-1-16,Albert Pujols
88119,pujolal01,2002,1,SLN,NL,157,590,118,185,40,...,NaN,Jose Alberto,235.0,75.0,R,R,2001-04-02,2020-08-08,1980-1-16,Albert Pujols
88120,pujolal01,2003,1,SLN,NL,157,591,137,212,51,...,NaN,Jose Alberto,235.0,75.0,R,R,2001-04-02,2020-08-08,1980-1-16,Albert Pujols
88121,pujolal01,2004,1,SLN,NL,154,592,133,196,51,...,NaN,Jose Alberto,235.0,75.0,R,R,2001-04-02,2020-08-08,1980-1-16,Albert Pujols
88122,pujolal01,2005,1,SLN,NL,161,591,129,195,38,...,NaN,Jose Alberto,235.0,75.0,R,R,2001-04-02,2020-08-08,1980-1-16,Albert Pujols


(2220, 40)

555